In [2]:
# Import packages for building map
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
pbk.output_notebook()
from sodapy import Socrata
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes 
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource, CustomJS,
                          CustomJSFilter, GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

Loading BokehJS ...

In [3]:
# Load file with state geometry
contiguous_cat = gpd.read_file('divisions-administratives-v1r0-comarques-1000000-20210122.shp', csr="epsg:25831")
# Load file with each region's population
df_pobl = pd.read_excel("PoblacioComarques.xls")
# importar el nostre dataset
client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16774)
# Convert to pandas DataFrame
results_df =  gpd.GeoDataFrame.from_records(results)
results_df.head()

C:\Users\Daniel\anaconda3\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


,any,zona,dat,via,pk,nommun,nomcom,nomdem,f_morts,f_ferits_greus,...,d_superficie,d_tipus_via,d_titularitat_via,d_tracat_altimetric,d_vent,grupdialab,hor,gruphor,tipacc,tipdia
0,2010,Zona urbana,2010-01-25T23:33:00.000,SE,999999,CANOVES I SAMALUS,Valles Oriental,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NA,NA,"Calma, vent molt suau",Feiners,2333,Nit,Col.lisió de vehicles en marxa,dill-dij
1,2010,Carretera,2010-10-31T01:00:00.000,N-240,999,LLEIDA,Segria,Lleida,0,1,...,Sec i net,Carretera convencional,Estatal,Pla,"Calma, vent molt suau",CapDeSetmana,1,Nit,Sortida de la calcada sense especificar,dg
2,2010,Carretera,2010-05-17T15:27:00.000,N-II,7087,FORNELLS DE LA SELVA,Girones,Girona,1,0,...,Sec i net,Carretera convencional,Estatal,Rampa o pendent,"Calma, vent molt suau",Feiners,1527,Tarda,Col.lisió de vehicles en marxa,dill-dij
3,2010,Zona urbana,2010-08-21T22:30:00.000,SE,999999,BARCELONA,Barcelones,Barcelona,0,2,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NA,NA,"Calma, vent molt suau",CapDeSetmana,223,Nit,Col.lisió de vehicles en marxa,dis
4,2010,Zona urbana,2010-05-07T17:45:00.000,SE,999999,BADALONA,Barcelones,Barcelona,0,1,...,Sec i net,Via urbana( inclou carrer i carrer residencial),NA,NA,"Calma, vent molt suau",CapDeSetmana,1745,Tarda,Bolcada a la calcada,div


In [4]:
# num d'accidents en funció de la comarca i ordenats alfabèticament respecte del nom de la comarca
com = pd.value_counts(results_df['nomcom'])
com = com.to_frame() # convertir a Data Frame
com = com.reset_index() # introduir un index (0, 1, 2...)
com = com.sort_values("index") # la columna de velocitat es diu index, ordenar-la
com = com.rename(columns={"index": "Comarca", "nomcom": "#accidents"}) # canviar els noms correctes
Comar = gpd.GeoDataFrame(com).set_index("Comarca")
Comar = Comar.reset_index()
Comar.head()

,Comarca,#accidents
0,Alt Camp,146
1,Alt Emporda,563
2,Alt Penedes,397
3,Alt Urgell,187
4,Alta Ribagorca,30


In [5]:
geom = contiguous_cat[["NOMCOMAR", "geometry"]] # escollir les columnes: comarca i geometria de mcp
geom = geom.sort_values('NOMCOMAR') # ordenar alfabèticament
Geom = gpd.GeoDataFrame(geom).set_index("NOMCOMAR") # tornem a posar l'índex
Geom = Geom.reset_index()
Geom.head()

,NOMCOMAR,geometry
0,Alt Camp,"POLYGON ((373020.449 4590188.741, 372482.744 4..."
1,Alt Empordà,"MULTIPOLYGON (((513760.140 4662388.220, 513758..."
2,Alt Penedès,"POLYGON ((386593.300 4595896.200, 388616.100 4..."
3,Alt Urgell,"MULTIPOLYGON (((360800.100 4649174.000, 360624..."
4,Alta Ribagorça,"POLYGON ((327485.124 4721105.796, 327436.728 4..."


In [6]:
# seleccionar columna de geometries
geometry = Geom[['geometry']]
# ordenem alfabèticament i deixem bonic el dataset de poblacions
df_pobl=df_pobl.sort_values('comarca')
df_pobl=pd.DataFrame(df_pobl).set_index('comarca')

In [7]:
df_pobl=df_pobl.reset_index()
df_pobl.head()

,comarca,pobl,pobl_percent
0,Alt Camp,44168,0.0059
1,Alt Emporda,136981,0.0182
2,Alt Penedes,107724,0.0143
3,Alt Urgell,20201,0.0027
4,Alta Ribagorca,3813,0.0005


In [8]:
# seleccionar columnes pobl i pobl_percent
norm = df_pobl[['pobl', 'pobl_percent']]
combo = pd.concat([Comar, geometry, norm], axis=1) # juntem comarques, #accidents, geometria i poblacions
combo['accpercadamilhab'] = combo['#accidents']/combo['pobl']*1000 # afegim col per a #accidents per cada 1000 habitants
type(combo)

geopandas.geodataframe.GeoDataFrame

In [9]:
# Representacio interactiva
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = combo.to_json())

In [10]:
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '50': '500',
               '1000':'10000', '15000':'150000',
               '20000':'20000', '25000':'250000',
               '30000':'30000', '350000':'350000',
               '40000':'40000+'}

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = "Nombre d'accidents per cada 1000 habitants 2010-18", plot_height = 800 ,
           plot_width = 800, toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'accpercadamilhab', 'transform' : color_mapper},
                   line_color = 'gray', line_width = 0.25, fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Comarca','@Comarca'),('# Accidents', '@accpercadamilhab')]))

# Specify layout
p.add_layout(color_bar, 'below')

show(p)